In [128]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss

In [2]:
os.chdir('/aichallenge')

In [93]:
import time
t1=time.time()
train = pd.read_table('challengeData.tsv')
test = pd.read_table('scoring_set.tsv')
t2=time.time()
print(t2-t1)

/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,16,17,26,40,41,49,53,97,98,99,100,104,107,109,112,127,128,129,130,220,221,222) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,16,17,26,40,41,49,53,54,55,100,104,107,109,112,127,128,129,130,153,171,220,221,222,225,226,227) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


154.68219923973083


In [94]:
train_copy=train
test_copy=test

In [125]:
train=train_copy
test=test_copy

In [95]:
#remove data set name from columns names to make it uniform
t1=time.time()
columns=[]
for col in train.columns:
    columns.append(col.replace('challenge_data.',''))
train.columns=columns
columns=[]
for col in test.columns:
    columns.append(col.replace('scoring_set.',''))
test.columns=columns
t2=time.time()
print(t2-t1)

0.0012323856353759766


In [96]:
#remove the rows where 'challenge_data.renewed_yorn' is null in training data
t1=time.time()
train=train[train['renewed_yorn'].notnull()]
t2=time.time()
print(t2-t1)

0.35248327255249023


In [97]:
#remove the 'challenge_data.renewed_yorn' from training and test data and move it to a seperate variable
t1=time.time()
train_y=train['renewed_yorn']
train = train.drop('renewed_yorn',axis=1)
test_y=test['renewed_yorn']
test = test.drop('renewed_yorn',axis=1)
t2=time.time()
print(t2-t1)

3.120619773864746


In [98]:
#remove columns which have any null values
t1=time.time()
percent = (test.isnull().sum()/test.isnull().count())
test.columns[percent>0.5]
#We would ignore all these 102 columns so we are left with 135 columns
len(test.columns[percent>0.5])
#all_clean_columns=train.columns[percent==0]
#train_clean=train[all_clean_columns]
unclean_columns=test.columns[percent>0.5]
train = train.drop(unclean_columns,axis=1)
test = test.drop(unclean_columns,axis=1)
t2=time.time()
print(t2-t1)

46.19669699668884


In [99]:
len(unclean_columns)

36

In [100]:
unclean_columns

Index(['installation_date', 'service_contract_last_renewal_date',
       'service_contract_classification', 'contract_line_duration_in_days',
       'marketing_program_name', 'service_contract_discount_percentage',
       'pos_bookings_measure_transaction_type', 'pos_adjustment_type',
       'cisco_one_opportunity_type', 'ios_software_version',
       'service_offering', 'service_program', 'service_fee_amount',
       'last_date_of_support', 'end_new_service_attachment_date',
       'end_of_service_contract_renewal_date', 'end_software_maintenance_date',
       'service_level_group', 'mapped_service_list_price',
       'mapped_service_level', 'service_group', 'service_brand_code',
       'service_product_base_service_fee_amount',
       'service_product_base_last_date_of_support',
       'service_product_base_end_new_service_attachment_date',
       'service_product_base_end_of_service_contract_renewal_date',
       'service_product_base_end_software_maintenance_date',
       'service_

In [102]:
date_cols=['zip_code']
train=train.drop(date_cols,axis=1)
test=test.drop(date_cols,axis=1)

In [103]:
#Remove columns which have too many categories and is string type
t1=time.time()
messy_columns = []
for col in train.columns:
    if(train[col].dtype=='O' and len(train[col].unique())>20):
        messy_columns.append(col)
t2=time.time()
print(t2-t1)

2.4817826747894287


In [104]:
# 56 messy columns are removed
len(messy_columns)

64

In [105]:
messy_columns

['instance_id',
 'contract_line_id',
 'major_line_instance_id',
 'product_sales_order_type',
 'contract_line_start_date',
 'contract_line_end_date',
 'contract_line_create_date',
 'warranty_contract_line_start_date',
 'warranty_contract_line_end_date',
 'product_sales_channel_code',
 'previous_contract_line_end_date',
 'node_name',
 'level_3_name',
 'level_4_name',
 'level_5_name',
 'level_6_name',
 'service_sales_node_base_node_name',
 'service_sales_node_base_sub_scms_name',
 'service_sales_node_base_country_name',
 'service_sales_node_base_level_2_name',
 'service_sales_node_base_level_3_name',
 'service_sales_node_base_level_4_name',
 'service_sales_node_base_level_5_name',
 'service_sales_node_base_level_6_name',
 'product_name',
 'product_family',
 'business_unit',
 'product_type',
 'product_sub_group',
 'service_product_base_product_name',
 'service_product_base_product_family',
 'service_product_base_product_sub_group',
 'customer_name',
 'global_ultimate_name',
 'global_sub_ve

In [106]:
t1=time.time()
train=train.drop(messy_columns,axis=1)
test=test.drop(messy_columns,axis=1)
t2=time.time()
print(t2-t1)

2.642380475997925


In [107]:
train.shape

(226140, 135)

In [108]:
#Remove the primary key column for data fitting
t1=time.time()
train = train.drop('innovation_challenge_key',axis=1)
test = test.drop('innovation_challenge_key',axis=1)
t2=time.time()
print(t2-t1)

2.303266763687134


In [109]:
#Number of unique values in each columns
t1=time.time()
catcols=[]
for col in train.columns:
    if train[col].dtype=='O' and len(train[col].unique())<=20:
        catcols.append(col)
scalcols=['contract_line_net_usd_amount','product_net_price','sales_node_renewal_rate','partner_renewal_rate',
          'product_renewal_rate','service_sales_node_installed_base_sales_node_renewal_rate',
          'service_partner_installed_base_partner_renewal_rate']
t2=time.time()
print(t2-t1)

1.1244211196899414


In [110]:
for col in train_copy.columns:
    print(train_copy[col].dtype)

int64
object
object
object
object
object
object
object
object
object
int64
object
object
object
object
object
object
object
object
object
object
float64
object
object
object
object
object
object
float64
object
object
object
object
object
object
int64
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
int64
object
object
object
object
object
object
object
object
object
object
object
object
int64
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
float64
object
object
object
object
object
object
object
object
object
float64
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
float64
object
object
object
object
object
object
object
object
object
float64
float64
object
object
object
object
obje

In [111]:
train[catcols]=train[catcols].fillna('missing')
test[catcols]=test[catcols].fillna('missing')

In [112]:
#handle catcols
t1=time.time()
int_catcols=[]
for col in catcols:
    train[col]=train[col].astype('category')
    test[col]=test[col].astype('category')
t2=time.time()
print(t2-t1)

107.87235069274902


In [113]:
sum1=0
for col in train.columns:
    if train[col].dtype=='O':
        sum1=sum1+len(train[col].unique())
print(sum1)

0


In [114]:
for col in train.columns:
    print(col)
    print(train[col][0])

minor_line_yorn
Y
product_purchase_order_type
Resale
instance_status
Latest-INSTALLED
product_transaction_type
10002
installed_product_type
SERIALIZED
installed_product_component_yorn
Y
component_type
CHILD
scdc_product_yorn
Y
service_classification
TECHNICAL SERVICES
service_contract_process_type
missing
contract_line_multi_year_discount_yorn
N
contract_line_route_to_market_code
1-TIER
contract_line_sales_motion_code
RENEW
default_multi_year_discount_yorn
Y
low_dollar_contract_yorn
=
warranty_type
Standard Warranty
contract_line_source_business_process_name
UNKNOWN
contract_line_repair_service_order_level
UNKNOWN
contract_line_reaction_time_code
-1
product_bookings_measure_transaction_type
ERP
product_adjustment_type
UNKNOWN
product_channel_bookings_yorn
Y
product_route_to_market
DVAR
service_channel_bookings_yorn
Y
service_route_to_market_code
DVAR
service_bookings_measure_transaction_type
ERP
auto_quote_yorn
N
hardware_yorn
Y
software_yorn
N
acw_yorn
N
enterprise_license_agreement_y

In [115]:
train[scalcols]=train[scalcols].fillna(train[scalcols].mean())
test[scalcols]=test[scalcols].fillna(train[scalcols].mean())

In [116]:
train.columns[train.isnull().any()==True]

Index([], dtype='object')

In [117]:
#Normalize integer data
t1=time.time()
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler = scaler.fit(train[scalcols])
train[scalcols] = scaler.transform(train[scalcols])
scaler = scaler.fit(test[scalcols])
test[scalcols] = scaler.transform(test[scalcols])
t2=time.time()
print(t2-t1)

2.3735973834991455


In [118]:
train[scalcols].describe()

,contract_line_net_usd_amount,product_net_price,sales_node_renewal_rate,partner_renewal_rate,product_renewal_rate,service_sales_node_installed_base_sales_node_renewal_rate,service_partner_installed_base_partner_renewal_rate
count,2.261400e+05,2.261400e+05,2.261400e+05,2.261400e+05,226140.000000,2.261400e+05,2.261400e+05
mean,1.005455e-17,1.206546e-17,-8.524376e-17,-4.411435e-17,0.000000,5.031204e-17,-8.124158e-18
std,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002,1.000002e+00,1.000002e+00
min,-1.252316e-01,-2.321716e-01,-5.724046e+00,-3.235729e-01,-0.171298,-2.311610e+00,-6.788500e-02
25%,-1.252316e-01,-2.321716e-01,0.000000e+00,-6.406592e-03,0.000000,0.000000e+00,-1.663313e-02
50%,-1.252316e-01,-2.321716e-01,0.000000e+00,-5.496591e-17,0.000000,0.000000e+00,-8.882108e-18
75%,-1.252316e-01,-1.584198e-01,0.000000e+00,-5.496591e-17,0.000000,0.000000e+00,-8.882108e-18
max,7.774224e+01,6.648806e+01,7.628503e+01,1.612224e+02,196.387833,9.051475e+01,1.676407e+02


In [119]:
#convert data to one hot encoding to handle categorical values
t1=time.time()
train_objs_num = len(train)
dataset = pd.concat(objs=[train, test], axis=0)
dataset = pd.get_dummies(dataset)
train = dataset[:train_objs_num]
test = dataset[train_objs_num:]
t2=time.time()
print(t2-t1)

13.648167848587036


In [120]:
train.shape

(226140, 853)

In [24]:
#Replace all NaN left with 0
t1=time.time()
train = train.fillna(0)
test = test.fillna(0)
t2=time.time()
print(t2-t1)

0.28705453872680664


In [122]:
#check data quality now
train.isnull().any().any()

False

In [123]:
#check data quality now
test.isnull().any().any()

False

In [125]:
#split the data between train and validation set
t1=time.time()
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train,train_y,test_size=0.33, random_state=42)
t2=time.time()
print(t2-t1)

0.9893553256988525


In [126]:
#Dimensions of training data
X_train.shape

(151513, 853)

In [29]:
#Lets start fitting different models on the data
#1. start with simple Bayes model
t1=time.time()
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gaussian_model = gnb.fit(X_train,y_train)
t2=time.time()
print(t2-t1)

1.4430644512176514


In [30]:
# Lets see the prediction on our validation data
t1=time.time()
pred = gaussian_model.predict_proba(X_valid)
t2=time.time()
print(t2-t1)

0.5990414619445801


In [31]:
pred[1,]

array([1.15020444e-09, 9.99999999e-01])

In [32]:
train_y.head()

0    N
1    N
2    N
3    N
4    N
Name: renewed_yorn, dtype: object

In [33]:
from sklearn.metrics import log_loss
log_loss(y_valid,pred[:,1])

8.414693836815434

In [34]:
#find predicted value on test set
pred = gaussian_model.predict_proba(test_X)

In [39]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test['innovation_challenge_key']
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=pred[:,1]

In [40]:
os.chdir('/users/hdpsndbx125')

In [41]:
#very important to sort the values else won't be taken at submission
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')

In [42]:
#persist to a file
submit.to_csv('NaiveBayesGaussian_EntireTraining.csv',index=False)

In [43]:
submit.shape

(1888195, 2)

In [46]:
#fitting a multinomialNB
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=1.0, fit_prior=True)
clf.fit(X_train,y_train)
pred = clf.predict_proba(X_valid)
log_loss(y_valid,pred[:,1])

ValueError: Input X must be non-negative

In [ ]:
#find predicted value on test set
pred = clf.predict_proba(test_X)

In [47]:
from sklearn.naive_bayes import BernoulliNB
mnb = MultinomialNB(alpha=1.0)
mnb.fit(X_train,y_train)
pred = mnb.predict_proba(X_valid)
log_loss(y_valid,pred[:,1])

ValueError: Input X must be non-negative

In [133]:
from sklearn.linear_model import LogisticRegression as lr
t1=time.time()
lr1=lr(max_iter=300)
lr1.fit(X_train,y_train)
pred = lr1.predict_proba(X_valid)
print(log_loss(y_valid,pred[:,1]))
t2=time.time()
print(t2-t1)

0.5917643616204006
7.375692844390869


In [134]:
#find predicted value on test set
pred = lr1.predict_proba(test)

In [136]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test_copy['innovation_challenge_key']
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=pred[:,1]
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('LR_300-impute-missing.csv',index=False)

In [2]:
submit.shape

NameError: name 'submit' is not defined

In [137]:
from sklearn.linear_model import SGDClassifier
t1=time.time()
sgd = SGDClassifier(loss="log", penalty="l2",max_iter=200)
sgd.fit(X_train,y_train)
pred = sgd.predict_proba(X_valid)
print(log_loss(y_valid,pred[:,1]))
t2=time.time()
print(t2-t1)

10.670953238224957
61.806074142456055


/usr/lib64/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [44]:
print(log_loss(y_valid,pred[:,1]))

0.5705468084851013


In [139]:
#trying Random Forest
from sklearn.ensemble import RandomForestClassifier
t1=time.time()
rfc = RandomForestClassifier(max_depth=3, random_state=0)
rfc.fit(X_train, y_train)
pred = rfc.predict_proba(X_valid)
print(log_loss(y_valid,pred[:,1]))
t2=time.time()
print(t2-t1)

0.5847134596637497
2.1656336784362793


In [140]:
pred = rfc.predict_proba(test)

In [141]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test_copy['innovation_challenge_key']
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=pred[:,1]
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('RFC_depth4-missing.csv',index=False)

In [ ]:
from sklearn import svm
import time
t1=time.time()
svc= svm.SVC(kernel='rbf',probability=True,verbose=True)
svc.fit(X_train, y_train)
t2=time.time()
pred = svc.predict_proba(X_valid)
log_loss(y_valid,pred[:,1])

[LibSVM]

In [1]:
log_loss(y_valid,pred[:,1])

NameError: name 'log_loss' is not defined

In [152]:
clean_col_name=[]
for col in X_train.columns:
    col = col.replace('<','X')
    col = col.replace('>','Y')
    col = col.replace('[','A')
    col = col.replace(']','B')
    clean_col_name.append(col)
X_train.columns = clean_col_name
clean_col_name=[]
for col in X_valid.columns:
    col = col.replace('<','X')
    col = col.replace('>','Y')
    col = col.replace('[','A')
    col = col.replace(']','B')
    clean_col_name.append(col)
X_valid.columns = clean_col_name
clean_col_name=[]
for col in test.columns:
    col = col.replace('<','X')
    col = col.replace('>','Y')
    col = col.replace('[','A')
    col = col.replace(']','B')
    clean_col_name.append(col)
test.columns = clean_col_name

In [153]:
#Time for xgboost
import xgboost as xgb
import time
t1 = time.time()
model1 = xgb.XGBClassifier(max_depth=4)
#model2 = xgb.XGBClassifier(n_estimators=100, max_depth=8, learning_rate=0.1, subsample=0.5)
xgb1 = model1.fit(X_train, y_train)
#xgb2 = model2.fit(X_train.iloc[0:10,], y_train[0:10])
t2 = time.time()

In [155]:
t2-t1

228.5851128101349

In [154]:
xgb1

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [148]:
pred=xgb1.predict_proba(X_valid)

In [149]:
from sklearn.metrics import log_loss
log_loss(y_valid,pred[:,1])

0.5263551478728989

In [156]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test_copy['innovation_challenge_key']
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=xgb1.predict_proba(test)[:,1]
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('XGB-missing.csv',index=False)

In [57]:
os.listdir('.')

['XGB-entiretrainscl-allcol.csv',
 'LR_100iter.csv',
 'scoring_set.tsv',
 'LR_100-iter-all.csv',
 'training_set.tsv',
 'challengeData.tsv',
 'lost+found']

In [30]:
#Trying MLPerceptron
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(20,20,20))

In [33]:
t1=time.time()
mlp.fit(X_train,y_train)
t2=time.time()
print(t2-t1)

MemoryError: 

In [43]:
from sklearn.metrics import log_loss
pred = mlp.predict_proba(X_valid)
log_loss(y_valid,pred[:,1])

0.4238643172502673

In [33]:
#submit the prediction score
submit = pd.DataFrame()
submit['INNOVATION_CHALLENGE_KEY'] = test['innovation_challenge_key']

In [35]:
pred=mlp.predict_proba(test_X)

In [37]:
len(pred)

1888195

In [38]:
#RENEWAL_PROBABLIITY
submit['RENEWAL_PROBABLIITY']=pred[:,1]
submit=submit.sort_values('INNOVATION_CHALLENGE_KEY')
submit.to_csv('MLP_20_20_20_scl.csv',index=False)